<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte2Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [212]:
%%writefile Matrices.cpp
#include <iostream>
#include <cstdlib>
#include <thread>
#include <vector>
#include <random>
using namespace std;
#define MAX_DIMENSION 20
#define MIN_DIMENSION 5
#define MAX_VALOR 32
#define MIN_VALOR -32

void inicializarMatriz(int n, int ***matriz, int min, int max)
{
  (*matriz) = (int**)malloc(n*sizeof(int*));    
  random_device device;
  mt19937 generador(device());
  uniform_int_distribution<> distribucion(min, max);    
  for(int i=0; i<n; i++)
  {
    (*matriz)[i] = (int*)malloc(n*sizeof(int));
    for(int j=0; j<n; j++)          
      (*matriz)[i][j] = distribucion(generador);         
  }  
}

void calcularProducto(int n, int fila, int **matrizA, int **matrizB, int **matrizC)
{
  for(int i=0; i<n; i++)
    for(int j=0; j<n; j++)
      matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];       
}

void mostrarMatriz(int **matriz, int n, string nombre)
{
  cout<<endl<<"Matriz "<<nombre<<":"<<endl;
  for(int i=0; i<n; i++)
  {    
    cout<<"| ";
    for(int j=0; j<n; j++)
    {
      if(matriz[i][j] < -999)
        cout<<matriz[i][j]<<" ";
      else if (matriz[i][j] < -99 || matriz[i][j] > 999)
        cout<<" "<<matriz[i][j]<<" ";    
      else if (matriz[i][j] < -9 || matriz[i][j] > 99)
        cout<<"  "<<matriz[i][j]<<" "; 
      else if (matriz[i][j] < 0 || matriz[i][j] > 9)
        cout<<"   "<<matriz[i][j]<<" ";
      else if (matriz[i][j] < 10)
        cout<<"    "<<matriz[i][j]<<" ";                                  
    }       
    cout<<"  |"<<endl;
  }
}

int main(int argc, char *argv[])
{      
  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DE FILAS/COLUMNAS
  if( argc < 2 ) 
  {
    printf("Debe ingresar el numero de filas/columnas, dentro del rango de 5 a 20.\n");
    exit(1);
  }

  int n = atoi(argv[1]); //DIMENSIN DE LA MATRIZ    

  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DENTRO DE LOS LIMITES ESTABLECIDOS
  if( n < MIN_DIMENSION || n > MAX_DIMENSION ) 
  {
    printf("El numero de filas/columnas debe estar dentro del rango de 5 a 20.\n");
    exit(1);
  }    

  //CREO LAS Matrices
  int **A, **B, **CS, **CH;   

  //INICIALIZO LAS MATRICES, A Y B CON VALORES RANDOM, CS Y CH EN 0
  inicializarMatriz(n, &A, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &B, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &CS, 0, 0);
  inicializarMatriz(n, &CH, 0, 0);
                
  //CALCULO VALORES DE CS (SECUENCIAL)
  for(int i=0; i<n; i++)
    calcularProducto(n, i, A, B, CS);    
    
  //CALCULO VALORES DE CH (CON CONCURRENCIA)
  //CREO UN VECTOR PARA ALMACENAR LAS REFERENCIAS A LOS HILOS CREADOS
  vector<thread> hilos;

  //CREO N HILOS
  for(int i=0; i<n; i++)     
    hilos.push_back(thread(calcularProducto, n, i, A, B, CH));                 
           
  //ESPERO A LOS N HILOS 
  for(int i=0; i<n; i++) 
    hilos[i].join();

  //MUESTRO LOS RESULTADOS
  mostrarMatriz(A,n,"A");    
  mostrarMatriz(B,n,"B");
  mostrarMatriz(CS,n,"CS"); 
  mostrarMatriz(CH,n,"CH"); 

  return 0;
}

Overwriting Matrices.cpp


In [213]:
!g++ -pthread -o Matrices Matrices.cpp

In [216]:
!./Matrices 5


Matriz A:
|    15     8   -13    19    27   |
|   -31     1    30    -2     9   |
|     7   -13    -2   -15    -9   |
|   -15     9   -31   -19   -18   |
|    -3    26    10    11     3   |

Matriz B:
|    -3    27    13   -18     4   |
|   -32     3   -23    -7    -7   |
|   -32    10   -26   -24     1   |
|     3    24   -17    -2     3   |
|   -14   -15    -3    23     4   |

Matriz CS:
|  -206   350   -55   569   156   |
| -1031  -717 -1199    42   -71   |
|   540   -95   724  -164    36   |
|   944  -874   781   575  -283   |
| -1152   316 -1093  -321  -139   |

Matriz CH:
|  -206   350   -55   569   156   |
| -1031  -717 -1199    42   -71   |
|   540   -95   724  -164    36   |
|   944  -874   781   575  -283   |
| -1152   316 -1093  -321  -139   |
